# TTN-based 2D solidification prediction

### Authors

- [Lukas M. Koschmieder](mailto:l.koschmieder@access-technology.de), Access, Intzestraße 5, 52072 Aachen, Germany
- [Mahdi T. Rad](mailto:m.torabirad@access-technology.de), Access, Intzestraße 5, 52072 Aachen, Germany

In [ ]:
config = {
    # TTN REST API FQHN or IP address
    'hostname' : '127.0.0.1', 
    # TTN REST API port
    'port' : 9888, 
    # TTN REST API resource
    'resource' : '/predict', 
    # Show additional outputs
    'verbose' : True 
}

In [ ]:
%matplotlib inline
from ipywidgets import interactive
import json
import requests
import time
from IPython.display import clear_output
import ipywidgets as widgets
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot(x, y, u, v):
    with output:
        x, y, u, v = np.array(x), np.array(y), np.array(u), np.array(v)

        x_coords, y_coords = np.meshgrid(y, x)
        xy_coords = np.hstack((x_coords.flatten()[:,None], 
                               y_coords.flatten()[:,None]))
        
        u_grid = griddata(xy_coords, u.flatten(), 
                          (x_coords, y_coords), method='linear')
        v_grid = griddata(xy_coords, v.flatten(), 
                          (x_coords, y_coords), method='linear')

        fig, (ax1, ax2) = plt.subplots(2)

        ax1.set_title('Temperature')
        h1 = ax1.imshow(u_grid.T, interpolation='nearest', cmap=plt.cm.jet, 
                        origin='lower', aspect='equal', vmin=-1, vmax=0)
        cax1 = make_axes_locatable(ax1).append_axes("right", size="5%", pad=0.1)
        fig.colorbar(h1, cax=cax1)

        ax2.set_title('Solid fraction')
        h2 = ax2.imshow(v_grid.T, interpolation='nearest', cmap=plt.cm.jet, 
                        origin='lower', aspect='equal', vmin=0, vmax=1)
        cax2 = make_axes_locatable(ax2).append_axes("right", size="5%", pad=0.1)
        fig.colorbar(h2, cax=cax2)

        plt.tight_layout()
        plt.show()

def update(t):
    with output:
        rtt_start = time.time()
        try:
            respond = requests.get(server, data={ 't': t })
        except:
            print("ERROR: TTN server does not respond: {}".format(server))
            raise
        ui_rtt.value = str(time.time() - rtt_start)
        
        render_start = time.time()
        out = json.loads(respond.content)
        ui_predict.value = str(out['predict_time'])
        plot(out['x'], out['y'], out['u'], out['v'])
        ui_render.value = str(time.time() - render_start)    

server = "http://{hostname}:{port}{resource}".format_map(config)

ui_rtt = widgets.FloatText(description="RTT [s]", disabled=True)
ui_predict = widgets.FloatText(description="Prediction [s]", disabled=True)
ui_render = widgets.FloatText(description="Rendering  [s]", disabled=True)
ui_time = widgets.FloatSlider(description="Time [s]", min=0.0, max=1.0, 
                              step=0.01, continuous_update=False)

if 'verbose' in config and config['verbose']:
    display(ui_rtt, ui_predict, ui_render)

interactive_plot = interactive(update, t=ui_time)
output = interactive_plot.children[-1]
interactive_plot